In [54]:
import numpy as np
import functools 

In [2]:
import mnist_reader
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

In [3]:
y_train = y_train.reshape(len(y_train),1)

In [4]:
Y = ((y_train == np.array(range(10))).astype(int))
Y = Y.reshape(10,len(y_train))

In [5]:
def inflate_matrixes(flat_thetas, shapes):
    layers  = len(shapes)+1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)

    for i in range(layers-1):
        steps[i+1] = steps[i] + sizes[i]
    return[
        flat_thetas[steps[i]: steps[i+1]].reshape(*shapes[i])
        for i in range(layers-1)
    ]


In [46]:
def feed_forward(thetas, X): # Es sera de (784, 60000)
    a = [X]
    for i in range(len(thetas)):
        a.append(
            sigmoid(
                np.matmul( 
                    thetas[i], #Theta tendra (125,785) la primera vez 
                    np.vstack((#Este Vstack tiene (785, 60000)
                        np.ones(len(X_train[0])).reshape(1,len(X_train[0])),
                        a[i]
                    )) 
                )#La mutiplicacion deja (125,60000)
            )#LA sigmoide lo deja de (125,60000)
        )
    return a

In [6]:
def feed_forward(thetas, X):
    a = [X]
    for i in range(len(thetas)):
        a.append(
            sigmoid(
                np.matmul( thetas[i], a[i])
            )
        )
    return a

In [7]:
sigmoid = lambda x: 1.0/(1.0 + np.exp(-x))

In [8]:
X_train = X_train.T
y_train = (y_train.T).reshape(len(y_train),1)
a = [X_train]

In [9]:
theta0 = np.random.rand(125,784)
theta1 = np.random.rand(10,125)

In [10]:
thetas = [theta0, theta1]

In [11]:
shapes = [thetas[0].shape, thetas[1].shape]
shapes

[(125, 784), (10, 125)]

# PASO 1

In [12]:
triangulito1 = np.zeros(shapes[0], dtype = int)  
triangulito2 = np.zeros(shapes[1], dtype = int)  

In [13]:
deltaT = [triangulito1, triangulito2] 

In [14]:
layers = len(shapes)+1

# PASO 2.1 y 2.2

In [15]:
a = feed_forward(thetas, X_train)

In [16]:
a[0].shape

(784, 60000)

In [17]:
a[1].shape

(125, 60000)

In [18]:
a[2].shape

(10, 60000)

# PASO 2.3

In [19]:
deltas = [*range(layers-1), a[-1] - Y]

In [20]:
len(deltas)

3

In [21]:
deltas[2].shape

(10, 60000)

# PASO 2.4

In [22]:
for i in range(layers-2,0,-1):
    deltas[i] = (np.matmul(thetas[i].T,deltas[i+1])) * ((a[i])*(1-a[i]))

# PASO 2.5

In [23]:
for i in range(layers-1):
    deltaT[i] = deltaT[i] + deltas[i+1] @ a[i].T

In [24]:
len(deltaT)

2

In [25]:
_,m = X_train.shape
m

60000

In [36]:
D = deltaT
len(D)

2

In [28]:
for i in range(len(deltaT)):
    print(i)

0
1


In [30]:
deltaT[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
deltaT[0].shape

(125, 784)

In [32]:
deltaT[1].shape

(10, 125)

In [33]:
deltaT[1]

array([[54000., 54000., 54000., ..., 54000., 54000., 54000.],
       [54000., 54000., 54000., ..., 54000., 54000., 54000.],
       [54000., 54000., 54000., ..., 54000., 54000., 54000.],
       ...,
       [54000., 54000., 54000., ..., 54000., 54000., 54000.],
       [54000., 54000., 54000., ..., 54000., 54000., 54000.],
       [54000., 54000., 54000., ..., 54000., 54000., 54000.]])

In [34]:
np.true_divide(deltaT[1],m)

array([[0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
       ...,
       [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
       [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9]])

# PASO 3

In [37]:
for i in range(len(deltaT)):
    D[i] = np.true_divide(deltaT[i],m)

In [41]:
D[1].shape

(10, 125)

# PRUEBAS

In [35]:
shapes = [a.shape, b.shape]

m = a.flatten()
n = b.flatten()

c = np.array([*m, *n])

In [34]:
a =  np.arange(6).reshape(2, 3)
b =  np.arange(6,9).reshape(1, 3)

In [37]:
d = inflate_matrixes(c,shapes)

[array([[0, 1, 2],
       [3, 4, 5]]), array([[6, 7, 8]])]


In [55]:
flatten_list_of_arrays = lambda arrays: functools.reduce(lambda acc, v: np.array([*acc.flatten(),*v.flatten()]), arrays)

In [60]:
aa =  np.arange(4).reshape(2, 2)
bb =  np.arange(4,6).reshape(2, 1)
shapes = [aa.shape, bb.shape]

In [61]:
cc = [aa,bb]

In [62]:
dd = flatten_list_of_arrays(cc)

In [63]:
inflate_matrixes(dd, shapes)

[array([[0, 1],
        [2, 3]]),
 array([[4],
        [5]])]